<a href="https://colab.research.google.com/github/HussainMehdi/AGI-Assignment-Gamma/blob/main/Tutorial_5a.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q transformers accelerate bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 20.0 MB/s eta 0:00:00


In [2]:
#Initialize Pipeline and Load LLM

from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import torch

# Choose a free model (Falcon is lighter for Colab)
model_name = "tiiuae/falcon-7b-instruct"    # Invoking LLMs
# Alternative: "mistralai/Mistral-7B-Instruct-v0.1"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    load_in_8bit=True   # saves GPU memory
)

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer) # Entire Pipeline


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/281 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/9.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.48G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/117 [00:00<?, ?B/s]

Device set to use cuda:0


In [3]:
#Direct Prompting (Testing Pipeline with the model without any template)

query = "Why does crop health decay due to the weather?."    # Prompt

response = pipe(query, max_new_tokens=200, do_sample=True, temperature=0.7)   # Getting result from the Model.

print("\n[Direct Prompting Result]\n", response[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.



[Direct Prompting Result]
 Why does crop health decay due to the weather?.
There are different factors that contribute to crop health decay, but weather is one of the most significant factors. Crop health can be affected by weather conditions such as drought, heat, or cold. These conditions can impact the growth and development of the crop, leading to crop damage, lower yields, and crop failure. Crop health is also affected by environmental factors such as pests and diseases, as well as soil and water-related problems.


In [4]:
#Direct Prompting (Testing Pipeline with the model without any template)

query = "What is reason of dengue fever?."
response = pipe(query, max_new_tokens=200, do_sample=True, temperature=0.7)
print("\n[Direct Prompting Result]\n", response[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.



[Direct Prompting Result]
 What is reason of dengue fever?.
Dengue fever is caused by a virus that is transmitted through the bite of an infected mosquito. The virus is transmitted to humans when they are bitten by an infected mosquito, and can then spread through the mosquito's saliva when it bites another person.


In [5]:
#Direct Prompting (Testing Pipeline with the model without any template)

query = "What topic I should learn for agentic ai course?."
response = pipe(query, max_new_tokens=200, do_sample=True, temperature=0.7)
print("\n[Direct Prompting Result]\n", response[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.



[Direct Prompting Result]
 What topic I should learn for agentic ai course?.
The topics you should learn for an agentic AI course include natural language processing, machine learning, pattern recognition, and robotics. These are essential for developing an AI agent that can interact with humans and carry out complex tasks in a variety of environments.


In [6]:
query = "What topic I should learn for agentic ai course?."
response = pipe(query, max_new_tokens=200, do_sample=True, temperature=0.7)
print("\n[Direct Prompting Result]\n", response[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.



[Direct Prompting Result]
 What topic I should learn for agentic ai course?.
As an AI language model, I do not have personal preferences. However, some common topics for agentic AI courses include machine learning, deep learning, natural language processing, and robotics. It really depends on the specific course and the goals of the student.


In [8]:

 #Tool and Prompt Setup ( Calculator  is implemented then prompt is designed in such a way that it uses calculator as as tool )


# Define a calculator tool
def calculator_tool(task):
    try:
        return str(eval(task))
    except:
        return "Error"


# Ask a math question
user_query = "What is 4+2?"

#user_query = "What is weather?"

prompt = f"""
You are an AI agent. You can either:
 1. Answer directly using reasoning, OR
 2. Say: 'USE_CALCULATOR: <expression>' if it's math.


Question: {user_query}
"""

response = pipe(prompt, max_new_tokens=100)[0]['generated_text']
print("\n[Agent Decision]\n", response)

# Check if calculator is needed
if "USE_CALCULATOR:" in response:
    expr = response.split("USE_CALCULATOR:")[1].strip()
    result = calculator_tool(expr)
    print("\n[Calculator Used]\nResult:", result)
else:
    print("\n[AI Answer]\n", response)



Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.



[Agent Decision]
 
You are an AI agent. You can either:
1. Answer directly using reasoning, OR
2. Say: 'USE_CALCULATOR: <expression>' if it's math.


Question: What is 4+2?
Answer: 6.

4+2=?
The answer is 6.

[Calculator Used]
Result: Error


In [9]:
# Define a symptom checker tool
def symptom_checker_tool(query):
    data = {
        "fever": "Fever is a sign of infection. Monitor temperature and rest well.",
        "headache": "Headaches may be caused by dehydration, stress, or vision strain.",
        "cough": "Cough could be due to allergies or infection. Stay hydrated and avoid cold drinks.",
    }
    for key, value in data.items():
        if key in query.lower():
            return value
    return "No information found for that symptom."

# Define a prescription tool
def prescription_tool(condition):
    prescriptions = {
        "fever": "Paracetamol 500mg every 6 hours if needed, stay hydrated, and rest.",
        "headache": "Ibuprofen 200mg if no stomach issues, drink water, and take rest.",
        "cough": "Cough syrup (dextromethorphan) twice daily and warm fluids.",
    }
    for key, value in prescriptions.items():
        if key in condition.lower():
            return f"Prescription: {value}"
    return "No prescription available for that condition."

# Example query
user_query = "I have a headache, what medicine should I take?"

prompt = f"""
You are a helpful medical AI assistant.
You have access to two tools:
1. USE_SYMPTOM_TOOL: <query>  — for general symptom info.
2. USE_PRESCRIPTION_TOOL: <condition> — for medicine suggestions.

Decide which tool to use for the following question:
Question: {user_query}
"""

# Get response from model
response = pipe(prompt, max_new_tokens=100)[0]['generated_text']
print("\n[Agent Decision]\n", response)

if "USE_PRESCRIPTION_TOOL" in response:
    condition = user_query  # or extract condition
    result = prescription_tool(condition)
    print("\n[Prescription Tool Used]\nResult:", result)

elif "USE_SYMPTOM_TOOL" in response:
    query = user_query
    result = symptom_checker_tool(query)
    print("\n[Symptom Tool Used]\nResult:", result)

else:
    print("\n[AI Answer]\n", response)


Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.



[Agent Decision]
 
You are a helpful medical AI assistant.
You have access to two tools:
1. USE_SYMPTOM_TOOL: <query>  — for general symptom info.
2. USE_PRESCRIPTION_TOOL: <condition> — for medicine suggestions.

Decide which tool to use for the following question:
Question: I have a headache, what medicine should I take?
Answer: It depends on the cause and severity of your headache. You should consult with your doctor first to determine the best treatment.

[Prescription Tool Used]
Result: Prescription: Ibuprofen 200mg if no stomach issues, drink water, and take rest.


In [10]:
!pip install wikipedia


  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11678 sha256=5e24c22b28eb79db8ef57f83601b58405b87c0fb594d89d7779a2d2497d820d0
  Stored in directory: /root/.cache/pip/wheels/63/47/7c/a9688349aa74d228ce0a9023229c6c0ac52ca2a40fe87679b8
Successfully built wikipedia


In [11]:
import re, math, numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
from IPython.display import Image, display
import wikipedia
import google.generativeai as genai
import os


In [12]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import torch


model_name = "tiiuae/falcon-7b-instruct"

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Load model with offloading enabled
modelF = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",                       # automatically split across GPU/CPU
    load_in_8bit=True,                       # quantized mode
    llm_int8_enable_fp32_cpu_offload=True    # <--- enables CPU offload correctly
)

# Create text-generation pipeline
pipe = pipeline(
    "text-generation",
    model=modelF,
    tokenizer=tokenizer,
    torch_dtype=torch.float16,
    device_map="auto"
)



The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

`torch_dtype` is deprecated! Use `dtype` instead!
Device set to use cuda:0


**Tools**

In [13]:

# ---- Calculator Tool ----
def calculator_tool(task):
    try:
        expr = task.strip().replace("^", "**")
        return str(eval(expr, {"__builtins__": None}, math.__dict__))
    except Exception:
        return "Error"

# ---- Datetime Tool ----
def datetime_tool(task):
    now = datetime.now()
    task = task.lower()
    if any(word in task for word in ["date", "today", "day"]):
        return now.strftime("%Y-%m-%d")
    elif any(word in task for word in ["time", "now", "clock"]):
        return now.strftime("%H:%M:%S")
    return "Please specify 'date' or 'time'."


# ---- Weather Tool (mock) ----
def weather_tool(city):
    data = {
        "New York": "10°C, cloudy",
        "London": "12°C, light rain",
        "Delhi": "28°C, clear",
        "Tokyo": "18°C, windy"
    }
    return f"The weather in {city} is {data.get(city, 'unavailable')}."

# ---- Wikipedia Tool ----
def wikipedia_tool(query):
    try:
        return wikipedia.summary(query, sentences=2)
    except Exception:
        return "No result found."

# ---- Plot Tool ----
def plot_tool(expression, start=-10, end=10):
    try:
        x = np.linspace(start, end, 400)
        y = eval(expression, {"x": x, "np": np, "__builtins__": None})
        plt.figure(figsize=(6,4))
        plt.plot(x, y, label=f"y = {expression}")
        plt.title(f"Plot of {expression}")
        plt.xlabel("x")
        plt.ylabel("y")
        plt.legend()
        plt.grid(True)
        plt.savefig("plot_result.png")
        plt.close()
        display(Image("plot_result.png"))
        return "✅ Plot generated and displayed below."
    except Exception as e:
        return f"Error plotting: {e}"

# ---- Tool Router ----
def tool_router(response_text):
    tool_patterns = {
        "CALCULATOR": calculator_tool,
        "DATETIME": datetime_tool,
        "WEATHER": weather_tool,
        "WIKIPEDIA": wikipedia_tool,
        "PLOT": plot_tool
    }

    m = re.search(r"USE_TOOL:\s*([A-Z]+)\s*(.*)", response_text.strip())
    if not m:
        return None

    tool_name, argument = m.groups()
    tool_func = tool_patterns.get(tool_name)

    if tool_func:
        print(f"[🧩 Tool Invoked: {tool_name}] with argument → {argument.strip()}")
        result = tool_func(argument.strip())
        return {"tool": tool_name, "result": result}
    return None



Using Instruct Model

In [14]:
def run_agent(pipe, user_query):
    prompt = f"""
You are an AI assistant with access to these tools:
- CALCULATOR: for math (e.g., 2+2)
- DATETIME: for date/time queries
- WEATHER: for city weather info
- WIKIPEDIA: for factual summaries
- PLOT: to visualize math functions (e.g., x**2)

If the question requires using a tool, you MUST respond ONLY with:
USE_TOOL: <TOOL_NAME> <query>

Do not write anything else.

Otherwise, answer directly.

Question: {user_query}
"""

    # Generate output using the pipeline
    response = pipe(prompt, max_new_tokens=150, do_sample=True)
    response_text = response[0]["generated_text"].split("Question:")[-1].strip()

    print("\n[🤖 Model Output]\n", response_text)

    tool_result = tool_router(response_text)
    if tool_result:
        print(f"[Tool Result]: {tool_result['result']}")
    else:
        print("[Direct Answer]", response_text)


In [15]:

run_agent(pipe, "What is weather of karachi?")


Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.



[🤖 Model Output]
 What is weather of karachi?
Answer:
Karachi Weather
The current weather in Karachi, Pakistan is:

Crisp and chilly
Temperature: 21°C (68°F)
Humidity: 45%
Wind: 17 km/h
Timezone: +5 hours

You can use the tools I provided to find more specific weather info:

CALCULATOR:
DATERMOT:
WEATHER:
WIKIPEDIA:
PLOT:

<p>I hope you find this helpful. Good luck with your project!</p>
[Direct Answer] What is weather of karachi?
Answer:
Karachi Weather
The current weather in Karachi, Pakistan is:

Crisp and chilly
Temperature: 21°C (68°F)
Humidity: 45%
Wind: 17 km/h
Timezone: +5 hours

You can use the tools I provided to find more specific weather info:

CALCULATOR:
DATERMOT:
WEATHER:
WIKIPEDIA:
PLOT:

<p>I hope you find this helpful. Good luck with your project!</p>


In [16]:
 run_agent(pipe, "What is 3 * 3?")


Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.



[🤖 Model Output]
 What is 3 * 3?
Answer: 12


3 * 3 = 12
[Direct Answer] What is 3 * 3?
Answer: 12


3 * 3 = 12


In [17]:
run_agent(pipe, "Where is Karachi?.")


Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.



[🤖 Model Output]
 Where is Karachi?.
<p><strong>USE TOOL: Google Maps</strong></p>
<p>Answer:</p>
<p>Karachi is a port city located on the south coast of the Arabian Sea. It is the capital of the province of Sindh in the country of Pakistan. In terms of geography, it is positioned at the intersection of the Arabian Sea and the Indus River, which separates it from the Sindh and Baluchistan areas. It is a bustling city with a population of around 8 million people.</p>
[Direct Answer] Where is Karachi?.
<p><strong>USE TOOL: Google Maps</strong></p>
<p>Answer:</p>
<p>Karachi is a port city located on the south coast of the Arabian Sea. It is the capital of the province of Sindh in the country of Pakistan. In terms of geography, it is positioned at the intersection of the Arabian Sea and the Indus River, which separates it from the Sindh and Baluchistan areas. It is a bustling city with a population of around 8 million people.</p>


In [18]:
 run_agent(pipe, "Plot x**2 + 3*x + 2")


Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.



[🤖 Model Output]
 Plot x**2 + 3*x + 2
I'm sorry, I cannot plot the expression. The expression is not in a format that can be plotted. Please try again.
[Direct Answer] Plot x**2 + 3*x + 2
I'm sorry, I cannot plot the expression. The expression is not in a format that can be plotted. Please try again.
